In [1]:
%matplotlib notebook

In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [12]:
!jupyter nbextension list

Known nbextensions:
  config dir: /Users/tyler/.jupyter/nbconfig
    notebook section
      jupyter-gmaps/extension  enabled 
      - Validating: OK
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
  config dir: /Users/tyler/opt/anaconda3/etc/jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
      jupyter-gmaps/extension  enabled 
      - Validating: OK


In [13]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [14]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Paracelis,PH,17.2667,121.4667,85.57,60,90,4.45
1,1,Bambous Virieux,MU,-20.3428,57.7575,80.01,94,20,6.91
2,2,Rikitea,PF,-23.1203,-134.9692,79.07,69,11,14.54
3,3,Salalah,OM,17.0151,54.0924,77.00,47,0,1.14
4,4,Tasiilaq,GL,65.6145,-37.6368,30.20,93,90,18.41


In [15]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [16]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [17]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [18]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
print(fig)

Figure(layout=FigureLayout(height='420px'))


In [20]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [26]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Paracelis,PH,17.2667,121.4667,85.57,60,90,4.45
1,1,Bambous Virieux,MU,-20.3428,57.7575,80.01,94,20,6.91
2,2,Rikitea,PF,-23.1203,-134.9692,79.07,69,11,14.54
3,3,Salalah,OM,17.0151,54.0924,77.00,47,0,1.14
9,9,Victoria,HK,22.2855,114.1577,80.01,74,87,10.00
17,17,Sur,OM,22.5667,59.5289,79.97,61,2,5.48
20,20,Lai,TD,9.3952,16.2989,76.08,14,48,5.84
21,21,Alofi,NU,-19.0595,-169.9187,84.20,74,40,13.80
22,22,Yulara,AU,-25.2406,130.9889,80.60,22,0,16.11
23,23,Hilo,US,19.7297,-155.0900,75.20,73,90,4.61


In [27]:
preferred_cities_df.count()

City_ID       160
City          160
Country       160
Lat           160
Lng           160
Max Temp      160
Humidity      160
Cloudiness    160
Wind Speed    160
dtype: int64

In [28]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Paracelis,PH,85.57,17.2667,121.4667,
1,Bambous Virieux,MU,80.01,-20.3428,57.7575,
2,Rikitea,PF,79.07,-23.1203,-134.9692,
3,Salalah,OM,77.00,17.0151,54.0924,
9,Victoria,HK,80.01,22.2855,114.1577,
17,Sur,OM,79.97,22.5667,59.5289,
20,Lai,TD,76.08,9.3952,16.2989,
21,Alofi,NU,84.20,-19.0595,-169.9187,
22,Yulara,AU,80.60,-25.2406,130.9889,
23,Hilo,US,75.20,19.7297,-155.0900,


In [37]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [38]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [39]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Paracelis,PH,85.57,17.2667,121.4667,
1,Bambous Virieux,MU,80.01,-20.3428,57.7575,
2,Rikitea,PF,79.07,-23.1203,-134.9692,
3,Salalah,OM,77.00,17.0151,54.0924,
9,Victoria,HK,80.01,22.2855,114.1577,
...,...,...,...,...,...,...
562,Gaur,NP,85.26,26.7667,85.2667,
564,Amethi,IN,87.91,26.1500,81.8167,
565,Samarai,PG,84.09,-10.6167,150.6667,
567,Sangueya,GN,75.79,10.7000,-14.3667,


In [40]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [41]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [46]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [47]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))